In [1]:
#general imports
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import sklearn
import scipy.optimize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC 
import random
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

#fmclp imports
from fmclp import fmclp, cuae, synthetic_dataset, general_results_write, write_res, benefit, run_experiment

In [2]:
d = pd.read_csv('bar_pass_prediction.csv')

In [3]:
del d['ID']
del d['race1']
del d['race2']
del d['sex']
del d['bar']
del d['dnn_bar_pass_prediction']
del d['pass_bar']
del d['indxgrp2']
del d['gender']
del d['grad']
del d['Dropout']
del d['fulltime']
del d['lsat']
del d['zfygpa']
del d['ugpa']
del d['zgpa']
del d['other']
del d['asian']
del d['black']
del d['hisp']

In [4]:
def grouper_race(x):
    if x == 7:
        return 1
    else: 
        return 0
d['race'] = d['race'].apply(grouper_race)
def grouper_gpa(x):
    if x>3.4:
        return 2
    elif x <3.1:
        return 0
    else:
        return 1
d = d.dropna(how = 'any')
d['gpa'] = d['gpa'].apply(grouper_gpa)
d['sex_race']= d[['male','race']].apply(sum,axis=1).apply(lambda x: min(1,x))

In [5]:
del d['male']
del d['race']

In [6]:
del d['bar1']
del d['bar1_yr']
del d['bar2']
del d['bar2_yr']

In [7]:
d.columns

Index(['decile1b', 'decile3', 'decile1', 'cluster', 'DOB_yr', 'fam_inc', 'age',
       'parttime', 'bar_passed', 'tier', 'index6040', 'indxgrp', 'gpa',
       'sex_race'],
      dtype='object')

In [8]:
d = d.rename(columns = {'gpa':'target', 'sex_race':'attr'})
d = pd.get_dummies(d,drop_first = True)

In [9]:
y = d.drop(['target'], axis=1) 
x = d['target']
y_train,y_test,x_train,x_test = train_test_split(y,x)
lg = LGBMClassifier()
lg.fit(y_train, x_train)
lg = lg.predict(y_test)
accuracy_score(lg, x_test)

0.852359506559624

In [10]:
d.bar_passed.value_counts()

True     19389
False     1038
Name: bar_passed, dtype: int64

In [11]:
d.head()

,decile1b,decile3,decile1,cluster,DOB_yr,fam_inc,age,parttime,bar_passed,tier,index6040,target,attr,indxgrp_b 400-460,indxgrp_c 460-520,indxgrp_d 520-580,indxgrp_e 580-640,indxgrp_f 640-700,indxgrp_g 700+
0,10.0,10.0,10.0,1.0,69.0,5.0,-62.0,0.0,True,4.0,886.842082,2,1.0,0,0,0,0,0,1
1,5.0,4.0,5.0,2.0,69.0,4.0,-62.0,0.0,True,2.0,649.999987,2,1.0,0,0,0,0,1,0
2,3.0,2.0,3.0,3.0,65.0,1.0,-58.0,0.0,True,3.0,760.526298,2,1.0,0,0,0,0,0,1
3,7.0,4.0,7.0,3.0,58.0,4.0,-51.0,0.0,True,3.0,807.894717,2,1.0,0,0,0,0,0,1
4,9.0,8.0,9.0,4.0,68.0,4.0,-61.0,0.0,True,5.0,949.999974,2,1.0,0,0,0,0,0,1


In [12]:
d.columns

Index(['decile1b', 'decile3', 'decile1', 'cluster', 'DOB_yr', 'fam_inc', 'age',
       'parttime', 'bar_passed', 'tier', 'index6040', 'target', 'attr',
       'indxgrp_b 400-460', 'indxgrp_c 460-520', 'indxgrp_d 520-580',
       'indxgrp_e 580-640', 'indxgrp_f 640-700', 'indxgrp_g 700+'],
      dtype='object')

In [13]:
%%time
experiment = run_experiment(dataset = d,
                   number_experiments = 100,
                   multiplier = 50,
                   number_iterations = 20,
                   interior_classifier = 'rf',
                   folder = 'gpa_race_sex',
                   dataset_name='gpa_race_sex',
                   initial_classifier = LGBMClassifier())      

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
CPU times: user 4d 1h 40min 31s, sys: 55min 6s, total: 4d 2h 35min 38s
Wall time: 6h 13min 54s


In [15]:
unfair_downgraded= np.array( 
[0.15909091, 0.16438356, 0.16589862, 0.1465721  ,0.17974684 ,0.17431193,
 0.17067308, 0.12826603, 0.1956044 , 0.17078652 ,0.16901408 ,0.16933638,
 0.18324607, 0.16861827, 0.20813397, 0.16289593 ,0.15492958 ,0.17633929,
 0.16099773, 0.16312057, 0.16262136, 0.1691974 , 0.16820276 ,0.17865429,
 0.14189189, 0.16513761, 0.18043478, 0.1954023 , 0.18552036 ,0.18390805,
 0.17050691, 0.15207373, 0.15144231, 0.16853933, 0.16326531 ,0.16743119,
 0.13488372, 0.16350711, 0.18508997, 0.1728972 , 0.17449664 ,0.14320988,
 0.14489311, 0.1714922 , 0.16284404, 0.15165877, 0.16216216 ,0.165,
 0.12826603, 0.13800905, 0.18387909, 0.14420804, 0.15827338 ,0.16121495,
 0.1511335 , 0.16546763, 0.1655481 , 0.14380531, 0.16326531 ,0.18552036,
 0.14864865, 0.16461916, 0.18289786, 0.17857143, 0.15492958 ,0.16209476,
 0.18925831, 0.14180929, 0.16099773, 0.1937799 , 0.16129032 ,0.18981481,
 0.15058824, 0.14573991, 0.16063348, 0.15124153, 0.16256158 ,0.171875,
 0.16475973, 0.17974684, 0.16083916, 0.12910798, 0.15873016 ,0.16627078,
 0.18475751, 0.16589862 ,0.18544601, 0.1862069 , 0.16336634 ,0.15456674,
 0.15550756, 0.17283951, 0.15046296, 0.15346535, 0.17002237 ,0.16099773,
 0.14797136, 0.15502183, 0.16083916, 0.20043573])

fair_downgraded=np.array(
[0.13131313, 0.13013699, 0.12903226 ,0.11111111, 0.15189873, 0.12844037,
 0.14903846,0.10926366 ,0.14945055 ,0.14606742 ,0.14788732 ,0.13501144,
 0.12303665, 0.1264637 , 0.17703349, 0.13574661 ,0.12676056 ,0.13839286,
 0.13378685, 0.14184397, 0.12621359, 0.14533623 ,0.14285714 ,0.16241299,
 0.12837838, 0.14220183, 0.14782609, 0.17011494 ,0.14705882 ,0.13793103,
 0.12442396, 0.11751152, 0.11057692, 0.14157303 ,0.11564626 ,0.13990826,
 0.10697674, 0.11848341, 0.13881748, 0.16588785 ,0.15212528 ,0.11604938,
 0.09976247, 0.11358575, 0.13302752 ,0.14218009, 0.13267813 ,0.15,
 0.10926366, 0.11312217, 0.13602015 ,0.09456265 ,0.12709832 ,0.14719626,
 0.12342569, 0.13189448, 0.15212528, 0.12168142, 0.12244898 ,0.14705882,
 0.11486486, 0.13759214, 0.15914489, 0.13095238, 0.12676056 ,0.11970075,
 0.15601023, 0.10513447, 0.12471655, 0.15789474, 0.1359447  ,0.16203704,
 0.10823529, 0.12331839, 0.12217195 ,0.11060948, 0.12807882 ,0.140625,
 0.1395881 , 0.15189873, 0.15151515, 0.11032864 ,0.13378685 ,0.15914489,
 0.15473441, 0.14516129, 0.1314554 , 0.16321839, 0.11633663 ,0.12880562,
 0.13822894, 0.15802469, 0.12731481, 0.1039604 , 0.1409396  ,0.138322,
 0.11694511, 0.13973799, 0.10955711, 0.14814815])

In [16]:
fair_downgraded.mean(),unfair_downgraded.mean()

(0.1338810424, 0.16495532859999998)